In [1]:
import pandas as pd
import numpy as np
import pickle
import xlrd
import os

In [2]:
# PATH WHERE THE FILES ARE SAVED
databricks_path = './' #r'/dbfs/FileStore/tables/' # databricks file system
azure_path = None #'/mnt/ddf/' # Azure Blob Storage

In [3]:
if azure_path is not None:
    dbutils.fs.cp(azure_path + '/1_raw_data/framework_configuration.xlsx', databricks_path.replace('/dbfs/','dbfs:/'))

In [4]:
# FRAMEWORK CONFIGURATION
wb = pd.ExcelFile(databricks_path+ r'/1_raw_data/framework_configuration.xlsx')
ddf_config_var = wb.parse('Variables')
ddf_config_kpi = wb.parse('KPI')
ddf_config_par = wb.parse('Parameters')

# ddf_config_var = pd.read_excel(path + r'framework_configuration_var.xlsx')
# ddf_config_kpi = pd.read_excel(path + r'\temp\1_raw_data\framework_configuration_kpi.xlsx')
# ddf_config_par = pd.read_excel(path + r'framework_configuration_par.xlsx')

In [5]:
# VALIDATION OF THE EXISTENCE OF THE FILES IN THE FOLDER
data_source = ddf_config_var.loc[1,'DataSource']
if azure_path is not None:
    dbutils.fs.cp(azure_path + '/1_raw_data/' + data_source, databricks_path.replace('/dbfs/','dbfs:/'))

file_exist = os.path.isfile(databricks_path + "/1_raw_data/" + data_source)
if file_exist:
    ddf = pd.read_excel(databricks_path + "/1_raw_data/" + data_source)
else:
    print('The file does not exist or does not have the established name')

In [6]:
# PRIMARY KEY
primary_key = ddf_config_var.loc[np.where(ddf_config_var["VariableUsage"]=='PRIMARY')[0][0],'VariableName']

# FILTER BY COLUMN
filter_by = ddf_config_par.loc[0,'Value']

# FILTER BY VALUE
filter_value = ddf_config_par.loc[1,'Value']

if str(filter_by)!='nan':
    # PRIMARY KEY LIST WITH FILTER 
    pk_list = ddf[primary_key][ddf[filter_by]==filter_value].unique()
else:
    # PRIMARY KEY LIST 
    pk_list = ddf[primary_key].unique()

column_name = ddf_config_var.loc[np.where(ddf_config_var["VariableType"]=='NUMERIC')[0],'VariableName']

In [7]:
time_variables = list(ddf_config_var.loc[np.where(ddf_config_var["VariableType"]=='DATETIME')[0],'VariableName'])
numeric_variables = list(ddf_config_var.loc[np.where(ddf_config_var["VariableType"]=='NUMERIC')[0],'VariableName'])
categorical_variables = list(ddf_config_var.loc[np.where(ddf_config_var["VariableType"]=='CATEGORICAL')[0],'VariableName'])

# VARIABLES FORMAT
ddf[time_variables] = ddf[time_variables].astype('object')
for col in categorical_variables:
    ddf[col] = ddf[col].astype('category')
ddf[numeric_variables] = ddf[numeric_variables].astype('float')

# NUMBER OF MISSING VALUES BY VARIABLE
num_nan = ddf.isnull().sum()

# NUMBER OF INVALID VALUES BY VARIABLE
inv_values_aux = list()
for i in numeric_variables:
    inv_values_aux.append(ddf[i][np.logical_or(ddf[i]>float(ddf_config_var['UpperBound'][ddf_config_var['VariableName']==i]),ddf[i]<float(ddf_config_var['LowerBound'][ddf_config_var['VariableName']==i]))].count())
inv_values = pd.DataFrame([numeric_variables, inv_values_aux]).transpose()
inv_values.columns = ['numeric_variable','invalid_values']

'''
# REPLACEMENT OF INVALID VALUES BY THE CORRESPONDING LIMIT
for cname in numeric_variables:
    ddf[cname][ddf[cname]>float(ddf_config_var['UpperBound'][ddf_config_var['VariableName']==cname])] = float(ddf_config_var['UpperBound'][ddf_config_var['VariableName']==cname])
    ddf[cname][ddf[cname]<float(ddf_config_var['LowerBound'][ddf_config_var['VariableName']==cname])] = float(ddf_config_var['LowerBound'][ddf_config_var['VariableName']==cname])
'''

# REPLACEMENT OF INVALID VALUES BY NAN
for cname in numeric_variables:
    ddf[cname][ddf[cname]>float(ddf_config_var['UpperBound'][ddf_config_var['VariableName']==cname])] = np.nan
    ddf[cname][ddf[cname]<float(ddf_config_var['LowerBound'][ddf_config_var['VariableName']==cname])] = np.nan

# OUTLIER FACTOR
factor = 3

outliers = []
outliers_name = []

for i in numeric_variables:
    # DEFINITION OF LOWER AND UPPER LIMITS OF OUTLIERS CONSIDERING CRITERIA IQR BY PRIMARY KEY
    locals()['iq_{0}'.format(i[:3])] = pd.DataFrame(ddf.groupby([primary_key])[i].quantile(.25))
    locals()['iq_{0}'.format(i[:3])].columns = ['p25']
    locals()['iq_{0}'.format(i[:3])]['p75'] = ddf.groupby([primary_key])[i].quantile(.75)
    locals()['iq_{0}'.format(i[:3])]['iqr'] = locals()['iq_{0}'.format(i[:3])]['p75']-locals()['iq_{0}'.format(i[:3])]['p25']
    locals()['iq_{0}'.format(i[:3])]['lo_lim'] = locals()['iq_{0}'.format(i[:3])]['p25']-factor*locals()['iq_{0}'.format(i[:3])]['iqr']
    locals()['iq_{0}'.format(i[:3])]['up_lim'] = locals()['iq_{0}'.format(i[:3])]['p75']+factor*locals()['iq_{0}'.format(i[:3])]['iqr']
    # NUMBER OF OUTLIERS
    locals()['outliers_{0}'.format(i[:3])] = sum(ddf[i]<ddf.merge(locals()['iq_{0}'.format(i[:3])],left_on=primary_key,right_index=True)['lo_lim'].sort_index())+sum(ddf[i]>ddf.merge(locals()['iq_{0}'.format(i[:3])],left_on=primary_key,right_index=True)['up_lim'].sort_index())
    outliers.append(locals()['outliers_{0}'.format(i[:3])])
    outliers_name.append("outliers_" + i[:3])
    # REPLACEMENT OF OUTLIERS BY NAN
    ddf[i][ddf[i]<ddf.merge(locals()['iq_{0}'.format(i[:3])],left_on=primary_key,right_index=True)['lo_lim'].sort_index()] = np.nan
    ddf[i][ddf[i]>ddf.merge(locals()['iq_{0}'.format(i[:3])],left_on=primary_key,right_index=True)['up_lim'].sort_index()] = np.nan 

outlier_df = pd.DataFrame(outliers_name)
outlier_df.columns = ['outlier_name']
outlier_df['outlier_number'] = outliers

# IMPUTATION OF DATA MISSING AND OUTLIERS BY MEAN BY PRIMARY KEY
ddf[numeric_variables] = ddf.groupby(primary_key).transform(lambda x: x.fillna(x.mean()))
outlier_df

C:\Users\NW314MQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\NW314MQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\NW314MQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\NW314MQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipyk

,outlier_name,outlier_number
0,outliers_Vol,0
1,outliers_Rev,0
2,outliers_Pre,0


In [8]:
# EXPORTATION OF CLEAN DATA
writer = pd.ExcelWriter(databricks_path + r'/2_cleaned_data/clean_ddf.xlsx')
ddf.to_excel(writer,'Sheet1')
writer.save()


# EXPORT OF VARIABLES
preprocess1 = (databricks_path + r'/3_variables/var1_pre_processing.sav')
pickle.dump((num_nan, inv_values), open(preprocess1, 'wb'))

preprocess2 = (databricks_path + r'/3_variables/var2_pre_processing.sav')
pickle.dump((outlier_df), open(preprocess2, 'wb'))

frameworkconfig1 = (databricks_path + r'/3_variables/var1_framework_config.sav')
pickle.dump(ddf_config_var, open(frameworkconfig1, 'wb'))

frameworkconfig2 = (databricks_path + r'/3_variables/var2_framework_config.sav')
pickle.dump(ddf_config_kpi, open(frameworkconfig2, 'wb'))

frameworkconfig3 = (databricks_path + r'/3_variables/var3_framework_config.sav')
pickle.dump(ddf_config_par, open(frameworkconfig3, 'wb'))

In [9]:
if azure_path is not None:
    dbutils.fs.cp(databricks_path.replace('/dbfs/','dbfs:/') + "/clean_ddf.xlsx", azure_path + '/2_cleaned_data/clean_ddf.xlsx')